In [ ]:
   
# crops rectangular nan data added during reprojection
def Crop_Nans(data):

    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(data[0,:,:])[0]):

        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            print("F",lmi)
            break
        for lmj in range(np.shape(data[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
                    sx=lmi


            if(sy==0):
                if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(data[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(data[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
                
    print(sx,ex,sy,ey)
    return sx,ex,sy,ey


In [ ]:
# Return an input cube matched to the given nosie
from astropy import units as u  
from astropy.convolution import Gaussian1DKernel
from astropy.utils import NumpyRNGContext




def gaussian_beam(f, beam_gauss_width):
    '''
    Fourier transform of a Gaussian beam. NOT the power spectrum (multiply exp
    argument by 2 for power spectrum).
    Parameters
    ----------
    f : np.ndarray
        Frequencies to evaluate beam at.
    beam_gauss_width : float
        Beam size. Should be the Gaussian rms, not FWHM.
    '''
    return np.exp(-f**2 * np.pi**2 * 2 * beam_gauss_width**2)

    
#make a guassian correlation function
def gauss_correlated_noise_2D(shape, sigma, beam_gauss_width,
                              randomseed=327485749):
    
    '''
    Generate correlated Gaussian noise with sigma, smoothed by a
    Gaussian kernel.
    '''

    # Making a real signal. Only need real part of FFT
    freqs_yy, freqs_xx = np.meshgrid(np.fft.fftfreq(shape[0]),
                                     np.fft.rfftfreq(shape[1]), indexing="ij")

    freqs = np.sqrt(freqs_yy**2 + freqs_xx**2)
    # freqs[freqs == 0.] = np.NaN
    # freqs[freqs == 0.] = 1.

    imsize = shape[0]

    Np1 = (imsize - 1) // 2 if imsize % 2 != 0 else imsize // 2
    
    with NumpyRNGContext(randomseed):

        angles = np.random.uniform(0, 2 * np.pi,
                                   size=freqs.shape)

    noise = np.cos(angles) + 1j * np.sin(angles)

    if imsize % 2 == 0:
        noise[1:Np1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1, -1] = np.conj(noise[imsize:Np1:-1, -1])
        noise[Np1, 0] = noise[Np1, 0].real + 1j * 0.0
        noise[Np1, -1] = noise[Np1, -1].real + 1j * 0.0

    else:
        noise[1:Np1 + 1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1 + 1, -1] = np.conj(noise[imsize:Np1:-1, -1])

    # Zero freq components must have no imaginary part to be own conjugate
    noise[0, -1] = noise[0, -1].real + 1j * 0.0
    noise[0, 0] = noise[0, 0].real + 1j * 0.0

    corr_field = np.fft.irfft2(noise *
                               gaussian_beam(freqs, beam_gauss_width))

    norm = (np.sqrt(np.sum(corr_field**2)) / np.sqrt(corr_field.size)) / sigma

    corr_field /= norm
    
    return corr_field





def Noise_matching(Input_Cube,m,manual_noise=0*u.K):
    
    datn = Input_Cube.hdu.data

    npixels = np.product(Input_Cube.hdu.data.shape)
    
    
    #######
    #######
    # Calculate the RMS noise of the cube (for the line)
    #######
    #######

    # 2d or 3d?
    if len(np.shape(Input_Cube)) ==3:
        D3=True
        D2=False
    elif len(np.shape(Input_Cube)) ==2:
        D2=True
        D3=False

    if D3:
            
        Non_nan=((datn[0]>0)  | (datn[0]<0 )) # All the data that is not a nan value in the first (emissionless) channel of the cube
        
        m_current = np.nanstd(datn[0],where= Non_nan)*Input_Cube[0][0][0].unit #Noise (K)
        
        if manual_noise != 0*u.K:
            m_current = manual_noise
            
        actual_noise = m_current
        
        print("Current RMS noise found to be:",actual_noise,"Default to manual?: ",manual_noise,"Match to ",m)
        
        actual_noise=actual_noise.value
        target_noise = m.value
        
        additional_sigma = np.sqrt(np.abs(target_noise**2 - actual_noise**2))
        
        additional_noise = np.random.normal(0., additional_sigma, npixels)
        additional_noise = np.reshape(additional_noise, Input_Cube.hdu.data.shape)
        
        
        fwhm_factor = np.sqrt(8*np.log(2))
        add_noise = np.zeros(np.shape(datn))
        for lmi in range(len(datn)):
            new_seed = np.random.randint(1e9)

            ## use 1 pixel as beam size since there is no beam
            beam_gaussian_width = Input_Cube.header["CDELT1"]
            additional_noise = gauss_correlated_noise_2D(shape=(Input_Cube.hdu.data[6].shape[0],Input_Cube.hdu.data[6].shape[1]), sigma=additional_sigma, beam_gauss_width=beam_gaussian_width,randomseed=new_seed)
            pp=np.where(additional_noise!=np.nan)
            add_noise[lmi][pp]=additional_noise[pp]
        
        new_data = datn+add_noise
        QCopy = Input_Cube.hdu
        QCopy.data = new_data
        Q = SpectralCube.read(QCopy)
        del QCopy
        
        
        
        return Q
        
    if D2:
        
        Non_nan=((datn>0)  | (datn<0 )) # All the data that is not a nan value in the first (emissionless) channel of the cube
    
        m_current = np.nanstd(datn,where= Non_nan)*Input_Cube[0][0].unit #Noise (K)
        
        if manual_noise != 0*u.K:
            m_current = manual_noise
            
        actual_noise = m_current
        
        print("Current RMS noise found to be:",actual_noise,"Default to manual?: ",manual_noise,"Match to ",m)
        
        actual_noise=actual_noise.value
        target_noise = m.value
        
        additional_sigma = np.sqrt(np.abs(target_noise**2 - actual_noise**2))
    
        additional_noise = np.random.normal(0., additional_sigma, npixels)
        additional_noise = np.reshape(additional_noise, Input_Cube.hdu.data.shape)
    
        #create noise kernal
        fwhm_factor = np.sqrt(8*np.log(2))
        add_noise = np.zeros(np.shape(datn))
        new_seed = np.random.randint(1e9)
        additional_noise = gauss_correlated_noise_2D(shape=(Input_Cube.hdu.data[6].shape[0],Input_Cube.hdu.data[6].shape[1]), sigma=additional_sigma, beam_gauss_width=5/fwhm_factor,randomseed=new_seed)
        pp=np.where(additional_noise!=np.nan)
        add_noise[pp]=additional_noise[pp]

         #add noise
        new_data = datn+add_noise
        QCopy = Input_Cube.hdu
        QCopy.data = new_data
        Q = SpectralCube.read(QCopy)
        del QCopy
    
        
    
        return Q
        
        
        
     

In [4]:
#reshape the arrays for consistency
def make_256(arr=[],outsize = [256,256]):
    
    spread = [int(outsize[0]/2),int(outsize[1]/2)]
    midpoints = [int(len(arr[0])/2),int(len(arr[1])/2)]

    return  arr[midpoints[0]-spread[0]:midpoints[0]+spread[0],midpoints[1]-spread[1]:midpoints[1]+spread[1]]

#Function to pad the arrays if needed
def pad_to_256(arr=[], pad_value=0, outsize = [256,256]):
        
    h, w = arr.shape
    new_h, new_w = outsize[0], outsize[1]

    # compute padding needed
    pad_h = max(0, new_h - h)
    pad_w = max(0, new_w - w)

    # distribute padding equally before/after
    pad_top = pad_h // 2
    pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2
    pad_right = pad_w - pad_left

    # apply padding
    padded = np.pad(
        arr,
        ((pad_top, pad_bottom), (pad_left, pad_right)),
        mode="constant",
        constant_values=pad_value
    )

    return padded